# Joseph McInerney - Knowledge Engineering

In [1]:
# Set random seed to student number to allow for the reproduction of results.
random_seed = 40460549

In [2]:
!pip install -r Joseph_McInerney_40460549_requirements.txt

In [3]:
# Imported Libraries
import pandas as pd
from d3graph import d3graph, vec2adjmat # for ontology graph visual
from collections import deque #bfs

Load data

In [4]:
kg = pd.read_csv('./data/kg.csv')

C:\Users\josep\AppData\Local\Temp\ipykernel_11936\324444230.py:1: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  kg = pd.read_csv('./data/kg.csv')


# Part 1: Data Exploration

In [5]:
kg.head()

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,protein_protein,ppi,0,9796,gene/protein,PHYHIP,NCBI,8889,56992,gene/protein,KIF15,NCBI
1,protein_protein,ppi,1,7918,gene/protein,GPANK1,NCBI,2798,9240,gene/protein,PNMA1,NCBI
2,protein_protein,ppi,2,8233,gene/protein,ZRSR2,NCBI,5646,23548,gene/protein,TTC33,NCBI
3,protein_protein,ppi,3,4899,gene/protein,NRF1,NCBI,11592,11253,gene/protein,MAN1B1,NCBI
4,protein_protein,ppi,4,5297,gene/protein,PI4KA,NCBI,2122,8601,gene/protein,RGS20,NCBI


In [6]:
# fix bug where populating graph has problems when x type and name are the same, so just append _name.

In [7]:
# biological_process, molecular_function, cellular_component -> +_'name'
mapping = {
    'biological_process': 'biological_process_name',
    'molecular_function': 'molecular_function_name',
    'cellular_component': 'cellular_component_name'
}

# Apply the mapping to both columns
kg['x_name'] = kg['x_name'].replace(mapping)
kg['y_name'] = kg['y_name'].replace(mapping)

## 1.1 How many different types of relation, and how many of each type of relation?

In [8]:
# The number of different types of relation.
num_diff_relations = kg['relation'].nunique()
print(f'Number of different relations: {num_diff_relations}')

Number of different relations: 30


In [9]:
# The number of each type of relation there is.
count_per_relation = kg['relation'].value_counts()
print(count_per_relation)
print(f'Total: {count_per_relation.sum()}')

relation
anatomy_protein_present       3036406
drug_drug                     2672628
protein_protein                642150
disease_phenotype_positive     300634
bioprocess_protein             289610
cellcomp_protein               166804
disease_protein                160822
molfunc_protein                139060
drug_effect                    129568
bioprocess_bioprocess          105772
pathway_protein                 85292
disease_disease                 64388
contraindication                61350
drug_protein                    51306
anatomy_protein_absent          39774
phenotype_phenotype             37472
anatomy_anatomy                 28064
molfunc_molfunc                 27148
indication                      18776
cellcomp_cellcomp                9690
phenotype_protein                6660
off-label use                    5136
pathway_pathway                  5070
exposure_disease                 4608
exposure_exposure                4140
exposure_bioprocess              3250
exp

## 1.2 How many different types of nodes, and how many of each type are there? 

In [10]:
# The number of different types of node.
num_diff_nodes = kg['x_type'].nunique()
print(f'Number of different nodes: {num_diff_nodes}')
node_types = kg['x_type'].unique()
print(f'Nodes: {node_types}')

Number of different nodes: 10
Nodes: ['gene/protein' 'drug' 'effect/phenotype' 'disease' 'biological_process'
 'molecular_function' 'cellular_component' 'exposure' 'pathway' 'anatomy']


In [11]:
# inspection
node_counts = kg['x_type'].value_counts()
print(node_counts)

x_type
drug                  2805696
gene/protein          2631229
anatomy               1566154
disease                341244
effect/phenotype       257096
biological_process     252202
molecular_function      96723
cellular_component      93102
pathway                 47716
exposure                 9336
Name: count, dtype: int64


In [12]:
# Number of nodes
node_type_counts = kg.groupby('x_type')['x_index'].nunique().reset_index(name='count')
# sort
node_type_counts = node_type_counts.sort_values(by='count', ascending=False)
print(node_type_counts)
print(node_type_counts['count'].sum())

               x_type  count
1  biological_process  28642
7        gene/protein  27671
3             disease  17080
5    effect/phenotype  15311
0             anatomy  14035
8  molecular_function  11169
4                drug   7957
2  cellular_component   4176
9             pathway   2516
6            exposure    818
129375


In [13]:
# original paper: 129,375
print(f'Total Unique nodes: {kg['x_index'].nunique()}' )

Total Unique nodes: 129375


## 1.3 Comparison with Chanak 2023 "Building a knowledge graph to enable precision medicine"

In [14]:
# There are double the expected number of edges, explore why... directedness of edges?

## 1.4 For each type of node, sort the nodes of that type alphabetically by name, and list the index and name of the first three nodes in the sorted list.

In [15]:
# Sort the nodes by type and name
nodes_sorted = kg[['x_type', 'x_name']].sort_values(by=['x_type', 'x_name'])
# Get the first three nodes for each x_type
first_three_nodes = nodes_sorted.groupby('x_type').head(3)
# Keep original index, .to_string to fit wide df in output.
print(first_three_nodes[['x_type', 'x_name']].reset_index()[['x_type', 'index', 'x_name']].to_string(index=False))

            x_type   index                                                                      x_name
           anatomy 3840031                                               1st arch mandibular component
           anatomy 3842592                                                1st arch mandibular ectoderm
           anatomy 3842593                                                1st arch mandibular endoderm
biological_process 6161451                                          'de novo' AMP biosynthetic process
biological_process 6405428                                          'de novo' AMP biosynthetic process
biological_process 6405429                                          'de novo' AMP biosynthetic process
cellular_component 6194209                                          1,3-beta-D-glucan synthase complex
cellular_component 6194210                                          1,3-beta-D-glucan synthase complex
cellular_component 6195455                      1-alkyl-2-acetylglyceroph

## 1.5 Produce a table that shows how often each type of node is in each type of relation. 

In [16]:
counts_node_for_relation = kg.groupby(['relation', 'x_type']).size().reset_index(name='count')
counts_node_for_relation

,relation,x_type,count
0,anatomy_anatomy,anatomy,28064
1,anatomy_protein_absent,anatomy,19887
2,anatomy_protein_absent,gene/protein,19887
3,anatomy_protein_present,anatomy,1518203
4,anatomy_protein_present,gene/protein,1518203
5,bioprocess_bioprocess,biological_process,105772
6,bioprocess_protein,biological_process,144805
7,bioprocess_protein,gene/protein,144805
8,cellcomp_cellcomp,cellular_component,9690
9,cellcomp_protein,cellular_component,83402


# Part 2: Exploring the Knowledge Graph

## 2.1 Constructing and Visualising the Ontology

### Constructing

Now, add relations

In [17]:
# List to store (source, target, relation) triples for graph population
tuples = set()
# co-occurence frequency of nodes (x -> y), store type as well for bayesian inference later
joint_freq = {}
# source target dict for graph population later on
source_target = {}

# Go through rows of data set, constructing multiple data structure for other sections.
for row in kg.itertuples(index=False):
    #        ONTOLOGY
    #////////////////////////////////////////////////////////////
    # Get specifix x->y for that row (each row repersents one relation)
    x = row.x_name
    y = row.y_name

    x_type = row.x_type
    y_type = row.y_type
    # Append (source, target) thruple to tuples list, this is types as it is for the ontology
    tuples.add((x_type, y_type))
    #/////////////////////////////////////////////////////////////
    
    #         POPULATING KG 
    #////////////////////////////////////////////////////////////
    if x not in source_target:
        source_target[x] = []
    # when in Rome (iterrating through entier data set), may as well also get the actual nodes for population later
    source_target[x].append(y)
    #////////////////////////////////////////////////////////////
    
    #         BAYESIAN NETWORK
    #////////////////////////////////////////////////////////////
    # also for the bayesian network later on
    joint_freq[(x, y, x_type, y_type)] = joint_freq.get((x, y, x_type, y_type), 0) + 1
    #////////////////////////////////////////////////////////////


### Visualising

In [18]:
def GraphVis(tuples, fname): 
    # Recalculate source, target, and labels after deduplication
    source = []
    target = []
    labels = []
    edges_are_labeled = False
    # (source, target, relation) 
    for t in tuples:
        source.append(t[0])  
        target.append(t[1]) 
        if len(t) > 2:
            edges_are_labeled = True
            labels.append(t[2])

    # Adjacency matrix
    d3 = d3graph()
    adjmat = vec2adjmat(source, target)   

    d3.graph(adjmat)

    # Set edges to be directed
    if not edges_are_labeled:
        d3.set_edge_properties(
            directed=True, 
            marker_end='arrow', 
        )
    elif edges_are_labeled:
        d3.set_edge_properties(
            directed=True, 
            marker_end='arrow',
            label = labels
        )

    # Make graph more readable
    d3.set_node_properties(
        size = 8,
        color='#4682B4',
        fontcolor='red'
    )
    
    d3.show(filepath=fname)

    return None

GraphVis(tuples, './vis-ex.html')

[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Converting adjacency matrix into source-target..
[d3graph] INFO> Number of unique nodes: 10
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Number of unique nodes: 10
[d3graph] INFO> Slider range is set to [0, 1]
[d3graph] INFO> Write to path: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]
[d3graph] INFO> File already exists and will be overwritten: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]


## 2.2 Constructing the Knowledge Graph

Populate the KB with the data, nodes and edges 

#### Populate Graph

##### Nodes

In [19]:
# TheGraph: {node: ([relations], type)}
node_names = kg['x_name']
TheGraph = dict.fromkeys(node_names, None)

# zip() combines into pairs
nodes_types_dict = dict(zip(kg['x_name'], kg['x_type']))

# For all nodes in data
for name in node_names:
    type_corresponding_to_node = nodes_types_dict[name]
    # Initialise empty list to later add relations
    TheGraph[name] = ([], type_corresponding_to_node)

Now that the nodes have been added, we can connect the nodes. 

##### Edges

In [20]:
# Add List of relations for each node, retrieve information from sorce_target(s) dictionary
for source in source_target:
    targets = source_target.get(source) 
    # empty temp 'relations'
    relations, node_type = TheGraph[source]
    # populate relations
    relations = targets
    TheGraph[source] = (relations, node_type)  

## 2.3 Finding a Subgraph

In [21]:
# Generate subgraph, where root and target node are separated by 2 nodes, root node does not have loads of connections,
# - and that max search depth is 6.

In [22]:
def generate_subgraph_with_relations(TheGraph):
    # Get nodes of type 'disease'
    disease_nodes = [node for node, (_, node_type) in TheGraph.items() if node_type == 'disease']
        
    # take choose the first disease node as the root, #69 bidirectional relation
    for root_disease in disease_nodes:
        # Initialise subgraph
        subgraph = {}
        
        # BFS
        visited = set()
        queue = [(root_disease, 0)]
    
        # while still nodes availible to visit
        while queue:
            current_node, depth = queue.pop(0)
    
            # if new node
            if current_node not in visited:
                visited.add(current_node)
                relations, node_type = TheGraph[current_node]

                if len(relations) > 6:
                    break
                
                # Add new node to subgraph
                subgraph[current_node] = (relations, node_type)  
                
                # Add neigbours to q
                for neighbor in relations:
                    queue.append((neighbor, depth + 1))
                        
                # Stop searching if another disease node is found within 2 edges
                if depth == 3 and node_type == "disease" and current_node != root_disease:
                    target = current_node
                    print('Suitable subgraph found!!!')
                    return subgraph, root_disease, target

    print(visited)
    # return empty if no suitable subgraph found
    return {}, None, None


In [23]:
def extract_tuples_from_subgraph(subgraph):
    tuples = []
    for node, (relations, type_) in subgraph.items():
        for target in relations:
            tuples.append((node, target))
    return tuples

In [24]:
# generate suitable subgraph
subgraph, root, target = generate_subgraph_with_relations(TheGraph)

Suitable subgraph found!!!


In [25]:
subgraph

{'progressive peripheral pterygium': (['central pterygium',
   'conjunctival pterygium'],
  'disease'),
 'central pterygium': (['progressive peripheral pterygium'], 'disease'),
 'conjunctival pterygium': (['progressive peripheral pterygium',
   'conjunctival degeneration',
   'pterygium'],
  'disease'),
 'conjunctival degeneration': (['conjunctival pterygium',
   'pseudopterygium',
   'pinguecula',
   'conjunctival disorder'],
  'disease'),
 'pterygium': (['conjunctival pterygium',
   'familial pterygium of the conjunctiva',
   'IFNA2',
   'BICD2',
   'benign neoplasm of cornea',
   'benign conjunctival neoplasm'],
  'disease'),
 'pseudopterygium': (['conjunctival degeneration', 'corneal disease'],
  'disease')}

In [26]:
# format for visualisation
subgraph_tuples = extract_tuples_from_subgraph(subgraph)
print(subgraph_tuples)

[('progressive peripheral pterygium', 'central pterygium'), ('progressive peripheral pterygium', 'conjunctival pterygium'), ('central pterygium', 'progressive peripheral pterygium'), ('conjunctival pterygium', 'progressive peripheral pterygium'), ('conjunctival pterygium', 'conjunctival degeneration'), ('conjunctival pterygium', 'pterygium'), ('conjunctival degeneration', 'conjunctival pterygium'), ('conjunctival degeneration', 'pseudopterygium'), ('conjunctival degeneration', 'pinguecula'), ('conjunctival degeneration', 'conjunctival disorder'), ('pterygium', 'conjunctival pterygium'), ('pterygium', 'familial pterygium of the conjunctiva'), ('pterygium', 'IFNA2'), ('pterygium', 'BICD2'), ('pterygium', 'benign neoplasm of cornea'), ('pterygium', 'benign conjunctival neoplasm'), ('pseudopterygium', 'conjunctival degeneration'), ('pseudopterygium', 'corneal disease')]


In [27]:
# Visualise
GraphVis(subgraph_tuples, './subgraph-visual.html')

[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Converting adjacency matrix into source-target..
[d3graph] INFO> Number of unique nodes: 14
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Number of unique nodes: 14
[d3graph] INFO> Slider range is set to [0, 1]
[d3graph] INFO> Write to path: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\subgraph-visual.html]
[d3graph] INFO> File already exists and will be overwritten: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\subgraph-visual.html]


In [28]:
print(f'Root: {root}, Target: {target}')

Root: progressive peripheral pterygium, Target: pseudopterygium


# Part 3: Deriving a Knowledge Base and Inferring New Relations 

## 3.1 Knowledge Base (Rules) of Subgraph

In [29]:
# subgraph tuples contains a tuples of source -> target pairs which can also be understood as rules like p -> q
# rule form:  'if the node is x then the node is connected to y'
def get_rules_from_tuples(tuples):
    rules = []
    for tuple in tuples:
        x = tuple[0]
        y = tuple [1]
        rules.append(f'If the node is {x} then the node is connected to {y}')
    return rules

In [30]:
rules = get_rules_from_tuples(subgraph_tuples)
rules

['If the node is progressive peripheral pterygium then the node is connected to central pterygium',
 'If the node is progressive peripheral pterygium then the node is connected to conjunctival pterygium',
 'If the node is central pterygium then the node is connected to progressive peripheral pterygium',
 'If the node is conjunctival pterygium then the node is connected to progressive peripheral pterygium',
 'If the node is conjunctival pterygium then the node is connected to conjunctival degeneration',
 'If the node is conjunctival pterygium then the node is connected to pterygium',
 'If the node is conjunctival degeneration then the node is connected to conjunctival pterygium',
 'If the node is conjunctival degeneration then the node is connected to pseudopterygium',
 'If the node is conjunctival degeneration then the node is connected to pinguecula',
 'If the node is conjunctival degeneration then the node is connected to conjunctival disorder',
 'If the node is pterygium then the no

## 3.2 Subgraph Inference

In [31]:
def infer_transitive_relations(subgraph):
    # Initial explicit relations and reasoning
    inferred_tuples = set()  # Set of all inferred relations
    rule_reasoning = {}      # Chain of reasoning for each inferred relation

    # BFS queue
    queue = deque()
    for node, (relations, _) in subgraph.items():
        for neighbor in relations:
            inferred_tuples.add((node, neighbor))
            queue.append((node, neighbor))
            
    # BFS to infer new relations
    while queue:
        a, b = queue.popleft()  # Current relation (a → b)
        # Look for (b → c) to infer (a → c)
        if b in subgraph:
            for c in subgraph[b][0]:  # Neighbors of b
                if (a, c) not in inferred_tuples and a != c:
                    inferred_tuples.add((a, c))
                    queue.append((a, c))

                    # reasoning chain
                    rule_reasoning[(a, c)] = f"({a} → {b} AND {b} → {c}) IMPLIES {a} → {c}"

    return rule_reasoning


In [32]:
rules_and_reasons = infer_transitive_relations(subgraph)

In [33]:
for rule, reason in rules_and_reasons.items():
    print(f'Rule: {rule}')
    print(f'Reasoning: {reason}')
    print(f'\n')

Rule: ('progressive peripheral pterygium', 'conjunctival degeneration')
Reasoning: (progressive peripheral pterygium → conjunctival pterygium AND conjunctival pterygium → conjunctival degeneration) IMPLIES progressive peripheral pterygium → conjunctival degeneration


Rule: ('progressive peripheral pterygium', 'pterygium')
Reasoning: (progressive peripheral pterygium → conjunctival pterygium AND conjunctival pterygium → pterygium) IMPLIES progressive peripheral pterygium → pterygium


Rule: ('central pterygium', 'conjunctival pterygium')
Reasoning: (central pterygium → progressive peripheral pterygium AND progressive peripheral pterygium → conjunctival pterygium) IMPLIES central pterygium → conjunctival pterygium


Rule: ('conjunctival pterygium', 'central pterygium')
Reasoning: (conjunctival pterygium → progressive peripheral pterygium AND progressive peripheral pterygium → central pterygium) IMPLIES conjunctival pterygium → central pterygium


Rule: ('conjunctival pterygium', 'pseudo

# Part 4: A Bayesian View of the Data

## 4.1 Bayesian Network

In [34]:
# e joint distribution of anatomical region, protein, disease, and drug.

Visualize

In [35]:
nodes_for_baysian_network = ('disease', 'drug', 'gene/protein', 'anatomy')
JD_node_pairs = []
disease_incoming = []
for s,t in tuples:
    if s in nodes_for_baysian_network and t in nodes_for_baysian_network:
        JD_node_pairs.append((s,t))
    if t == 'disease':
        disease_incoming.append((s,t))

JD_node_pairs

[('disease', 'gene/protein'),
 ('anatomy', 'gene/protein'),
 ('drug', 'gene/protein'),
 ('gene/protein', 'drug'),
 ('disease', 'drug'),
 ('drug', 'drug'),
 ('gene/protein', 'disease'),
 ('gene/protein', 'anatomy'),
 ('disease', 'disease'),
 ('drug', 'disease'),
 ('gene/protein', 'gene/protein'),
 ('anatomy', 'anatomy')]

In [36]:
GraphVis(JD_node_pairs, './vis-ex.html')

[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Converting adjacency matrix into source-target..
[d3graph] INFO> Number of unique nodes: 4
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Number of unique nodes: 4
[d3graph] INFO> Slider range is set to [0, 1]
[d3graph] INFO> Write to path: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]
[d3graph] INFO> File already exists and will be overwritten: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]


Now I need to define the direction of these relations. 


In [37]:
BN_vis = []
BN_vis.append(('anatomy', 'gene/protein'))
BN_vis.append(('anatomy', 'gene/protein'))
BN_vis.append(('gene/protein', 'disease'))
BN_vis.append(('disease', 'drug'))

In [38]:
GraphVis(BN_vis, './vis-ex.html')

[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Converting adjacency matrix into source-target..
[d3graph] INFO> Number of unique nodes: 4
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Number of unique nodes: 4
[d3graph] INFO> Slider range is set to [0, 2]
[d3graph] INFO> Write to path: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]
[d3graph] INFO> File already exists and will be overwritten: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]


In [39]:
BN_vis_P = []
BN_vis_P.append(('anatomy', 'gene/protein', 0.9821))
BN_vis_P.append(('gene/protein', 'disease', 0.0306 ))
BN_vis_P.append(('disease', 'drug', 0.1174))

In [40]:
GraphVis(BN_vis_P, './vis-ex.html')

[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Converting adjacency matrix into source-target..
[d3graph] INFO> Number of unique nodes: 4
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Number of unique nodes: 4
[d3graph] INFO> Slider range is set to [0, 1]
[d3graph] INFO> Write to path: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]
[d3graph] INFO> File already exists and will be overwritten: [C:\Users\josep\OneDrive - Queen's University Belfast\Machine_Learning\ke-project\code\vis-ex.html]


## 4.2 Probabilistic Inference 

In [41]:
# joint_freq: frequency of (x,y pair and their type), co_occurence x, y, x_type, y_type
#for co_occurence in joint_freq.items():
    #freq = joint_freq[co_occurence[0]]
    #if freq > 2:
        #print(co_occurence)